In [1]:
import pandas as pd
import torch
from sentence_transformers import util

df = pd.read_csv('ramen_DB_final_embedded.csv',encoding='utf-8-sig')

df["openai_emb"] = df["openai_emb"].apply(lambda x: x[1:-1].split(', '))
df["openai_emb"] = df["openai_emb"].apply(lambda x: list(map(float, x)))

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def make_result(pref):
    temp_list = []

    for i in df.index:
        temp_ramen_name = df.loc[i, 'name']
        temp_emb = df.loc[i, 'openai_emb']

        if pref == '1_1':
            temp_df = df.copy()
        elif pref == '1_2':
            temp_df = pd.DataFrame(df[df['calories'] > df.loc[i, 'calories']])
        elif pref == '1_3':
            temp_df = pd.DataFrame(df[df['calories'] < df.loc[i, 'calories']])

        elif pref == '2_1':
            temp_df = pd.DataFrame(df[df['sodium'] > df.loc[i, 'sodium']])
        elif pref == '2_2':
            temp_df = pd.DataFrame(df[(df['sodium'] > df.loc[i, 'sodium']) & (df['calories'] > df.loc[i, 'calories'])])
        elif pref == '2_3':
            temp_df = pd.DataFrame(df[(df['sodium'] > df.loc[i, 'sodium']) & (df['calories'] < df.loc[i, 'calories'])])

        elif pref == '3_1':
            temp_df = pd.DataFrame(df[df['sodium'] < df.loc[i, 'sodium']])
        elif pref == '3_2':
            temp_df = pd.DataFrame(df[(df['sodium'] < df.loc[i, 'sodium']) & (df['calories'] > df.loc[i, 'calories'])])
        elif pref == '3_3':
            temp_df = pd.DataFrame(df[(df['sodium'] < df.loc[i, 'sodium']) & (df['calories'] < df.loc[i, 'calories'])])

        else:
            raise ValueError('Invalid pref')

        temp_df = temp_df.reset_index(drop=True)

        try:
            cos_scores = util.pytorch_cos_sim(temp_emb, temp_df['openai_emb'])[0]
        except:
            temp_list.append({})
            continue

        top_k = 10
        while top_k > 0:
            try:
                top_results = torch.topk(cos_scores, k=top_k)
                break
            except:
                top_k -= 1

        top_result_id = top_results.indices.tolist()
        top_result_score = top_results.values.tolist()

        temp_result_df = temp_df[temp_df.index.isin(top_result_id)]
        temp_result_df.index = top_result_id
        temp_result_df['score'] = top_result_score

        for j in temp_result_df.index:
            if (temp_ramen_name in temp_result_df.loc[j, 'name']) | (temp_result_df.loc[j, 'name'] in temp_ramen_name):
                temp_result_df = temp_result_df.drop(j)

        temp_result_df = temp_result_df.head(5)

        temp_result_df = temp_result_df[['name', 'sodium', 'calories']].to_dict('records')
        
        temp_list.append(temp_result_df)
    
    return temp_list

ValueError: buffer source array is read-only

In [9]:
# Sodium: I don't care
# Calories: I don't care

df_1_1 = df.copy()
df_1_1 = df_1_1[['name', 'sodium', 'calories']]

df_1_1['result'] = make_result('1_1')

df_1_1.to_json("./json/idc+idc.json", orient='records', force_ascii=False)
    

/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_49237/149084532.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_49237/149084532.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_49237/149084532.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [32]:
# Sodium: I don't care
# Calories: Higher

df_1_2 = df.copy()
df_1_2 = df_1_2[['name', 'sodium', 'calories']]

df_1_2['result'] = make_result('1_2')

df_1_2.to_json("./json/idc+higher.json", orient='records', force_ascii=False)

/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [33]:
# Sodium: I don't care
# Calories: Lower

df_1_3 = df.copy()
df_1_3 = df_1_3[['name', 'sodium', 'calories']]

df_1_3['result'] = make_result('1_3')

df_1_3.to_json("./json/idc+lower.json", orient='records', force_ascii=False)

/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [34]:
# Sodium: Higher
# Calories: I don't care

df_2_1 = df.copy()
df_2_1 = df_2_1[['name', 'sodium', 'calories']]

df_2_1['result'] = make_result('2_1')

df_2_1.to_json("./json/higher+idc.json", orient='records', force_ascii=False)

/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_result_df['score'] = top_result_score
/var/folders/y1/ph3w34z56n912fp0p8r0_1qh0000gn/T/ipykernel_48438/1586057505.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [4]:
# Sodium: Higher
# Calories: Higher

df_2_2 = df.copy()
df_2_2 = df_2_2[['name', 'sodium', 'calories']]

df_2_2['result'] = make_result('2_2')

df_2_2.to_json("./json/higher+higher.json", orient='records', force_ascii=False)

ValueError: buffer source array is read-only

In [7]:
# Sodium: Higher
# Calories: Lower

df_2_3 = df.copy()
df_2_3 = df_2_3[['name', 'sodium', 'calories']]

df_2_3['result'] = make_result('2_3')

df_2_3.to_json("./json/higher+lower.json", orient='records', force_ascii=False)

TypeError: unsupported operand type(s) for &: 'bool' and 'float'

In [ ]:
# Sodium: Lower
# Calories: I don't care

df_3_1 = df.copy()
df_3_1 = df_3_1[['name', 'sodium', 'calories']]

df_3_1['result'] = make_result('3_1')

df_3_1.to_json("./json/lower+idc.json", orient='records', force_ascii=False)

In [ ]:
# Sodium: Lower
# Calories: Higher

df_3_2 = df.copy()
df_3_2 = df_3_2[['name', 'sodium', 'calories']]

df_3_2['result'] = make_result('3_2')

df_3_2.to_json("./json/lower+higher.json", orient='records', force_ascii=False)

In [ ]:
# Sodium: Lower
# Calories: Lower

df_3_3 = df.copy()
df_3_3 = df_3_3[['name', 'sodium', 'calories']]

df_3_3['result'] = make_result('3_3')

df_3_3.to_json("./json/lower+lower.json", orient='records', force_ascii=False)